In [1]:
import sys
sys.path.append('/Users/clairemallard/Documents/Kitematic/underworld2_geodynamics/workspace/user_data/pygplates_rev18_python27_MacOS64')
import pygplates
print pygplates

<module 'pygplates' from '/Users/clairemallard/Documents/Kitematic/underworld2_geodynamics/workspace/user_data/pygplates_rev18_python27_MacOS64/pygplates.so'>


In [2]:
from __future__ import print_function
import math
import os

In [3]:
# Add directory containing the 'ptt' module (Plate Tectonic Tools) to the Python path.
sys.path.append('/Users/clairemallard/Documents/Kitematic/underworld2_geodynamics/workspace/user_data/PlateTectonicTools-master')
from ptt import ridge_spreading_rate


In [4]:
from ptt import separate_ridge_transform_segments


transform_segment_deviation_in_radians = separate_ridge_transform_segments.DEFAULT_TRANSFORM_SEGMENT_DEVIATION_RADIANS


rotation_model = pygplates.RotationModel('/Users/clairemallard/Downloads/GPlates/Matthews_etal_GPC_2016_410-0Ma_GK07.rot')
dynamic_polygons = ['/Users/clairemallard/Downloads/GPlates/Matthews_etal_GPC_2016_MesozoicCenozoic_PlateTopologies.gpmlz',
                    '/Users/clairemallard/Downloads/GPlates/Matthews_etal_GPC_2016_Paleozoic_PlateTopologies.gpmlz',
                    '/Users/clairemallard/Downloads/GPlates/Matthews_etal_GPC_2016_TopologyBuildingBlocks.gpmlz']



The area of the plates

What this does not do is:
- discriminate between trenches where the plate you are considering is the upper or lower plate
- anything to do with continents - of the five things you listed, 4 can be done directly from the resolved topology polygons without much trouble (pygplates is designed exactly for this), but as soon as you want to also look at the relationship between the resolved topologies and continents then a fair amount of extra work (input polygons and coding) is required.
- compute velocities

For the first point, there is a function within ‘PlateTectonicsTools’ called ‘find_overriding_and_subducting_plates’ which can do this, I just ran out of time figuring out how to include this in the attached notebook (maybe John can help with that)


In [5]:
reconstruction_time = 50.

resolved_topologies = []
resolved_topological_sections = []
pygplates.resolve_topologies(dynamic_polygons,
                             rotation_model,
                             resolved_topologies,
                             reconstruction_time,
                             resolved_topological_sections)


In [6]:
for resolved_topology in resolved_topologies:
    plate_id = resolved_topology.get_feature().get_reconstruction_plate_id()
    plate_area = resolved_topology.get_resolved_geometry().get_area() * pygplates.Earth.mean_radius_in_kms**2
    plate_perimeter = resolved_topology.get_resolved_geometry().get_arc_length() * pygplates.Earth.mean_radius_in_kms
    print plate_id , plate_area, plate_perimeter
    

SyntaxError: invalid syntax (<ipython-input-6-fc167a0fe0c7>, line 5)

In [ ]:
# Returns True if the specified resolved topology is the overriding plate at one of its subduction segment (also specified).
def is_overriding_plate(resolved_topology, resolved_topology_sub_segment):
    
    # Get the subduction polarity of the nearest subducting line.
    subduction_polarity = resolved_topology_sub_segment.get_feature().get_enumeration(pygplates.PropertyName.gpml_subduction_polarity)
    if (not subduction_polarity) or (subduction_polarity == 'Unknown'):
        # We don't know if it's the overriding plate, so just return false.
        return False
    
    sub_segment_reversal_flag = resolved_topology_sub_segment.was_geometry_reversed_in_topology()
    
    if resolved_topology.get_resolved_boundary().get_orientation() == pygplates.PolygonOnSphere.Orientation.clockwise:
        # The resolved boundary has clockwise orientation (when viewed from above the Earth).
        # If the overriding plate is to the 'left' of the subducting line (when following its vertices in order) and
        # the subducting line is reversed when contributing to the resolved boundary then the resolved boundary is the overriding plate.
        # A similar test applies to the 'right' but with the subducting line not reversed in the resolved boundary.
        if ((subduction_polarity == 'Left' and sub_segment_reversal_flag) or
            (subduction_polarity == 'Right' and not sub_segment_reversal_flag)):
            return True
        else:
            return False
    else:
        # The resolved boundary has counter-clockwise orientation (when viewed from above the Earth).
        # If the overriding plate is to the 'left' of the subducting line (when following its vertices in order) and
        # the subducting line is not reversed when contributing to the resolved boundary then the resolved boundary is the overriding plate.
        # A similar test applies to the 'right' but with the subducting line reversed in the topology.
        if ((subduction_polarity == 'Left' and not sub_segment_reversal_flag) or
            (subduction_polarity == 'Right' and sub_segment_reversal_flag)):
            return True
        else:
            return False


In [ ]:
# iterate over topological polygons
for resolved_topology in resolved_topologies:
    
    plate_id = resolved_topology.get_feature().get_reconstruction_plate_id()
    subduction_length_kms = 0
    subduction_overriding_length_kms = 0
    subduction_subducting_length_kms = 0
    ridge_length_kms = 0
    transform_length_kms = 0
    
    for sub_segment in resolved_topology.get_boundary_sub_segments():
        
        # Deal with subduction zones.....
        if sub_segment.get_resolved_feature().get_feature_type() == pygplates.FeatureType.gpml_subduction_zone:
            
            # See if resolved topology is the overriding plate at at the current subduction sub-segment.
            if is_overriding_plate(resolved_topology, sub_segment):
                subduction_overriding_length_kms += sub_segment.get_geometry().get_arc_length() * pygplates.Earth.mean_radius_in_kms
            else:
                subduction_subducting_length_kms += sub_segment.get_geometry().get_arc_length() * pygplates.Earth.mean_radius_in_kms
            
            subduction_length_kms += sub_segment.get_geometry().get_arc_length() * pygplates.Earth.mean_radius_in_kms

            
        # Deal with mid ocean ridge features (ridges + transforms)....
        elif sub_segment.get_resolved_feature().get_feature_type() == pygplates.FeatureType.gpml_mid_ocean_ridge:
            
            spreading_feature = sub_segment.get_feature()
            
            spreading_stage_rotation = separate_ridge_transform_segments.get_stage_rotation_for_reconstructed_geometry(
                                spreading_feature, rotation_model, reconstruction_time)
            if not spreading_stage_rotation:
                # Skip current feature - it's not a spreading feature.
                continue
                            
            ridge_and_transform_segment_geometries = separate_ridge_transform_segments.separate_geometry_into_ridges_and_transforms(
                                spreading_stage_rotation,
                                sub_segment.get_resolved_geometry(),
                                transform_segment_deviation_in_radians)
            if not ridge_and_transform_segment_geometries:
                # Skip shared sub segment - it's not a polyline (or polygon).
                continue

            ridge_sub_segment_geometries, transform_sub_segment_geometries = ridge_and_transform_segment_geometries
            
            for ridge_sub_segment_geometry in ridge_sub_segment_geometries:
                for segment in ridge_sub_segment_geometry.get_segments():
                    ridge_length_kms += segment.get_arc_length() * pygplates.Earth.mean_radius_in_kms
                        
            for transform_sub_segment_geometry in transform_sub_segment_geometries:
                for segment in transform_sub_segment_geometry.get_segments():
                    transform_length_kms += segment.get_arc_length() * pygplates.Earth.mean_radius_in_kms

    #break
    print plate_id, subduction_length_kms, subduction_overriding_length_kms, subduction_subducting_length_kms, \
            ridge_length_kms, transform_length_kms
    
    

In [7]:
# See if resolved topology is the overriding plate at the current subduction sub-segment.
    if is_overriding_plate(resolved_topology, sub_segment):
        subduction_overriding_length_kms += sub_segment.get_geometry().get_arc_length() * pygplates.Earth.mean_radius_in_kms
    else:
        subduction_subducting_length_kms += sub_segment.get_geometry().get_arc_length() * pygplates.Earth.mean_radius_in_kms

IndentationError: unexpected indent (<ipython-input-7-96bdf8416d17>, line 2)

github The area of the continents

I needed to know the ratio of continental to oceanic plate velocities in the Paleozoic. It does something similar to what you want, and also has the velocity aspect in there. 


In [ ]:
import pygplates
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd

import sys
sys.path.append('/Users/Simon/GIT/GPlatesClassStruggle/')
from reconstruction_classes import *
from create_gpml import create_gpml_healpix_mesh

import matplotlib as mpl
mpl.rcParams['font.family'] = "sans-serif"
mpl.rcParams['font.sans-serif'] = "arial"
mpl.rcParams['text.usetex'] == True

%matplotlib inline

#######################################################
# Define Input files for Muller 2016 AREPS model
#######################################################
# Specify the name of the GPlates rotation file



M2016 = ReconstructionModel('Matthews++2016')
M2016.add_rotation_model('../input_files/Rotations/Matthews_etal_GPC_2016_410-0Ma_GK07.rot')
M2016.add_static_polygons('../input_files/StaticPolygons/Muller_etal_AREPS_2016_StaticPolygons.gpml')
M2016.add_dynamic_polygons('../input_files/DynamicPolygons/Matthews_etal_GPC_2016_MesozoicCenozoic_PlateTopologies.gpml')
M2016.add_dynamic_polygons('../input_files/DynamicPolygons/Matthews_etal_GPC_2016_Paleozoic_PlateTopologies.gpml')
M2016.add_dynamic_polygons('../input_files/DynamicPolygons/Matthews_etal_GPC_2016_TopologyBuildingBlocks.gpml')
M2016.add_continent_polygons('../input_files/Global_Model_WD_Internal_Release_2016_v5/StaticGeometries/ContinentalPolygons/PresentDay_ContinentalPolygons_2016_v5.shp')


Y2018 = ReconstructionModel('Young++2018')
Y2018.add_rotation_model('../input_files/Global_Model_WD_Internal_Release_2016_v5/Global_EB_250-0Ma_GK07_2016_v5.rot')
Y2018.add_rotation_model('../input_files/Global_Model_WD_Internal_Release_2016_v5/Global_EB_410-250Ma_GK07_2016_v5.rot')
Y2018.add_static_polygons('../input_files/Global_Model_WD_Internal_Release_2016_v5/StaticGeometries/StaticPolygons/Global_EarthByte_GPlates_PresentDay_StaticPlatePolygons_2016_v5.shp')
Y2018.add_dynamic_polygons('../input_files/Global_Model_WD_Internal_Release_2016_v5/Global_EarthByte_Mesozoic-Cenozoic_plate_boundaries_2016_v5.gpml')
Y2018.add_dynamic_polygons('../input_files/Global_Model_WD_Internal_Release_2016_v5/Global_EarthByte_Paleozoic_plate_boundaries_2016_v5.gpml')
Y2018.add_dynamic_polygons('../input_files/Global_Model_WD_Internal_Release_2016_v5/TopologyBuildingBlocks_AREPS.gpml')
Y2018.add_continent_polygons('../input_files/Global_Model_WD_Internal_Release_2016_v5/StaticGeometries/ContinentalPolygons/PresentDay_ContinentalPolygons_2016_v5.shp')

In [ ]:
def velocity_versus_crustal_type(reconstruction_model, reconstruction_time):
    snapshot = reconstruction_model.plate_snapshot(reconstruction_time)
    velocity_field = snapshot.velocity_field()

    continent_partitioner = pygplates.PlatePartitioner(reconstruction_model.continent_polygons, 
                                                       reconstruction_model.rotation_model, reconstruction_time)

    multipoint_features = create_gpml_healpix_mesh(64)

    for f in multipoint_features:
        points = f.get_geometry().get_points()
        num_points = len(f.get_geometry().get_points().to_lat_lon_list())

    normalised_area_of_one_point = ((4.*np.pi) * pygplates.Earth.mean_radius_in_kms**2)/num_points

    points_in_continents = []
    for point in points:
        if continent_partitioner.partition_point(point):
            points_in_continents.append(point)

    points_in_continent_multipoint = pygplates.Feature()
    points_in_continent_multipoint.set_geometry(pygplates.MultiPointOnSphere(points_in_continents))

    plate_partitioner = pygplates.PlatePartitioner(reconstruction_model.dynamic_polygons, 
                                                   reconstruction_model.rotation_model, reconstruction_time)
    partitioned_continent_points = plate_partitioner.partition_features(points_in_continent_multipoint)


    #plt.plot(points_in_continent_multipoint.get_geometry().to_lat_lon_array()[:,1],
    #         points_in_continent_multipoint.get_geometry().to_lat_lon_array()[:,0],'.')
    #plt.show()

    res = []
    for plate,area in zip(snapshot.plate_ids,snapshot.plate_areas):

        area_of_continent_in_plate = 0
        for p in partitioned_continent_points:
            if p.get_reconstruction_plate_id()==plate:
                area_of_continent_in_plate += normalised_area_of_one_point * len(p.get_geometry().get_points().to_lat_lon_list())
        rms = velocity_field.rms_velocity(plate) 

        if area>0:
            res.append([reconstruction_time,plate,area,area_of_continent_in_plate,rms])
        else:
            print 'Zero area for plate %d at time %0.1fMa' % (plate,area)

    return res
    


res = velocity_versus_crustal_type(M2016, 120)

width=0.4
    
fig, ax = plt.subplots(figsize=(12,6))
plt.bar(np.arange(len(res)),zip(*res)[2],width=width,color='r')
plt.bar(np.arange(len(res))+width,zip(*res)[3],width=width,color='b')
ax.set_xticks(np.arange(len(res))+width/2)
ax.set_xticklabels((zip(*res)[1]),rotation=70)
plt.show()

In [ ]:
#resc = []
# Data frame template defining the column names
DataFrameTemplate = ('time','plate_id','plate_area','continental_area','rms_velocity')

# Create an empty dataframe to concatenate results to
df_M2016 = pd.DataFrame(columns=DataFrameTemplate)

for t in np.arange(0,401,1):
    #print t
    res = velocity_versus_crustal_type(M2016, t)
        
    df = pd.DataFrame(res,columns=DataFrameTemplate)
    
    #plt.plot(100*df['continental_area']/df['plate_area'],df['rms_velocity'],'o')
    
    df_M2016 = df_M2016.append(df)

    
# Create an empty dataframe to concatenate results to
df_Y2018 = pd.DataFrame(columns=DataFrameTemplate)

for t in np.arange(0,401,1):
    #print t
    res = velocity_versus_crustal_type(Y2018, t)
        
    df = pd.DataFrame(res,columns=DataFrameTemplate)
    
    df_Y2018 = df_Y2018.append(df)

In [ ]:
from scipy.stats import kde

xi, yi = np.mgrid[0:102:2, 0:305:5]

def velocity_versus_continental_fraction(df_AllTimes):
    area_threshold = 1.e6
    plt.figure(figsize=(12,6))
    for i,time in enumerate(np.arange(0,400,50)):
        plt.subplot(2,4,i+1)
        subset = df_AllTimes[(df_AllTimes['time']>=time) 
                             & (df_AllTimes['time']<time+50) 
                             & (df_AllTimes['plate_area']>area_threshold)]
        plt.plot(100*subset['continental_area']/subset['plate_area'],subset['rms_velocity'],'ko',markersize=5,alpha=0.1)

        subset.replace([np.inf, -np.inf], np.nan, inplace=True)
        subset.dropna(inplace=True)

        ratio = 100*subset['continental_area']/subset['plate_area']
        ratio[np.isnan(ratio)] = 0.

        k = kde.gaussian_kde(np.vstack((ratio,subset['rms_velocity'])))
        zi = k(np.vstack([xi.flatten(), yi.flatten()]))
        plt.contourf(xi, yi, zi.reshape(xi.shape), zorder=0, cmap=plt.cm.Blues)

        plt.ylim(0,250)
        plt.xlim(0,100)
        plt.grid()
        plt.title('%d - %d Ma' % (time,time+50))
        
        if i in [0,4]:
            plt.ylabel('RMS Velocity (mm/yr)')
        if i > 3:
            plt.xlabel('Continental %')
        
    plt.tight_layout()
    #plt.show()
        
        
velocity_versus_continental_fraction(df_M2016)
plt.savefig('../figures/RMS_Velocity_versus_percent_continent_M2016.png')
velocity_versus_continental_fraction(df_Y2018)
plt.savefig('../figures/RMS_Velocity_versus_percent_continent_Y2018.png')


In [ ]:
xi, yi = np.mgrid[0:1e8:1e6, 0:300:5]

plt.figure(figsize=(12,8))
for i,time in enumerate(np.arange(0,400,50)):
    plt.subplot(2,4,i+1)
    subset = df_M2016[(df_M2016['time']>=time) & (df_M2016['time']<time+50)]
    plt.plot(subset['plate_area'],subset['rms_velocity'],'o',alpha=0.2)
    
    subset.dropna(subset=['rms_velocity'], inplace=True)
    
    k = kde.gaussian_kde(np.vstack((subset['plate_area'],subset['rms_velocity'])))
    zi = k(np.vstack([xi.flatten(), yi.flatten()]))
    plt.contour(xi, yi, zi.reshape(xi.shape), zorder=3)
    
    plt.ylim(0,300)
    plt.xlim(0,1e8)
    plt.grid()
    plt.title('%d - %d Ma' % (time,time+50))
    
plt.figure(figsize=(12,8))
for i,time in enumerate(np.arange(0,400,50)):
    plt.subplot(2,4,i+1)
    subset = df_Y2018[(df_Y2018['time']>=time) & (df_Y2018['time']<time+50)]
    plt.plot(subset['plate_area'],subset['rms_velocity'],'o',alpha=0.2)
    
    subset.dropna(subset=['rms_velocity'], inplace=True)
    
    k = kde.gaussian_kde(np.vstack((subset['plate_area'],subset['rms_velocity'])))
    zi = k(np.vstack([xi.flatten(), yi.flatten()]))
    plt.contour(xi, yi, zi.reshape(xi.shape), zorder=3)
    
    plt.ylim(0,300)
    plt.xlim(0,1e8)
    plt.grid()
    plt.title('%d - %d Ma' % (time,time+50))

In [ ]:
import scipy.signal as sig

def velocity_versus_time(df_AllTimes):
    mean_oc = []
    mean_co = []
    mean_mixed = []
    mean_global = []

    for i,time in enumerate(np.arange(0,401,1)):
        subset = df_AllTimes[df_AllTimes['time']==time].dropna()

        subset_oc = subset[(subset['continental_area']/subset['plate_area'])<0.05]    
        mean_oc.append(np.average(subset_oc['rms_velocity'],weights=subset_oc['plate_area']))

        subset_co = subset[(subset['continental_area']/subset['plate_area'])>0.5]    
        mean_co.append(np.average(subset_co['rms_velocity'],weights=subset_co['plate_area']))

        subset_mixed = subset[((subset['continental_area']/subset['plate_area'])<0.5) & 
                              ((subset['continental_area']/subset['plate_area']))>0.05]    
        mean_mixed.append(np.average(subset_mixed['rms_velocity'],weights=subset_mixed['plate_area']))

        mean_global.append(np.average(subset['rms_velocity'],weights=subset['plate_area']))


    plt.figure(figsize=(6,4))
    plt.plot(np.arange(0,401,1),sig.medfilt(mean_oc,11),'royalblue',linewidth=3,label='>95% oceanic',zorder=3)
    plt.plot(np.arange(0,401,1),sig.medfilt(mean_co,11),'magenta',linewidth=3,label='>50% continental')
    plt.plot(np.arange(0,401,1),sig.medfilt(mean_mixed,11),'goldenrod',linewidth=3,label='other')
    plt.plot(np.arange(0,401,1),sig.medfilt(mean_global,11),'grey',linewidth=3,alpha=0.3,label='all plates')
    plt.xlim(400,0)
    plt.ylim(0,200)
    plt.grid()
    plt.legend()
    plt.xlabel('Time before present (Myr)',fontsize=12)
    plt.ylabel('RMS Velocity (mm/yr)',fontsize=12)
    #plt.show()

velocity_versus_time(df_M2016)
plt.savefig('../figures/RMS_Velocity_continents_versus_oceans_M2016.png')
velocity_versus_time(df_Y2018)
plt.savefig('../figures/RMS_Velocity_continents_versus_oceans_Y2018.png')

In [ ]:
#print df_M2016.plate_id.unique()

from collections import OrderedDict

target_plate_id_list = OrderedDict()

target_plate_id_list['901'] = {'color':'navy', 'name':'Pacific'}
target_plate_id_list['902'] = {'color':'blue', 'name':'Farallon'}
target_plate_id_list['911'] = {'color':'royalblue', 'name':'Nazca'}
target_plate_id_list['926'] = {'color':'dodgerblue', 'name':'Izanagi'}
target_plate_id_list['918'] = {'color':'lightblue', 'name':'Kula'}
target_plate_id_list['919'] = {'color':'steelblue', 'name':'Catequil'}
target_plate_id_list['908'] = {'color':'powderblue', 'name':'Chazca'}
target_plate_id_list['982'] = {'color':'skyblue', 'name':'Manihiki'}
target_plate_id_list['983'] = {'color':'slateblue', 'name':'Hikurangi'}
target_plate_id_list['16111'] = {'color':'green', 'name':'Alexander'}
target_plate_id_list['701'] = {'color':'goldenrod', 'name':'S Africa'}
target_plate_id_list['715'] = {'color':'yellow', 'name':'NE Africa'}
target_plate_id_list['801'] = {'color':'khaki', 'name':'Australia'}
target_plate_id_list['802'] = {'color':'burlywood', 'name':'Antarctica'}
target_plate_id_list['201'] = {'color':'tan', 'name':'S America'}
target_plate_id_list['501'] = {'color':'orange', 'name':'India'}
target_plate_id_list['101'] = {'color':'salmon', 'name':'N America'}
target_plate_id_list['301'] = {'color':'orangered', 'name':'Eurasia'}
#target_plate_id_list['102'] = {'color':'rosybrown', 'name':'Greenland'}
#target_plate_id_list['528'] = {'color':'hotpink', 'name':'Kohistan N'}
#target_plate_id_list['529'] = {'color':'lightpink', 'name':'Kohistan S'}
target_plate_id_list['530'] = {'color':'plum', 'name':'MesoTethys'}
target_plate_id_list['534'] = {'color':'mediumpurple', 'name':'NTethys'}
target_plate_id_list['53401'] = {'color':'mediumpurple', 'name':'Ind. NTethys'}
target_plate_id_list['605'] = {'color':'mediumpurple', 'name':'NTethys'}
target_plate_id_list['505'] = {'color':'orchid', 'name':'Cimm. NTethys'}
target_plate_id_list['380'] = {'color':'violet', 'name':'M-O'}
target_plate_id_list['9010'] = {'color':'violet', 'name':'Amurian M-O'}
target_plate_id_list['9009'] = {'color':'magenta', 'name':'Siberian M-O'}
target_plate_id_list['305'] = {'color':'olivedrab', 'name':'Variscan PTethys'}
target_plate_id_list['9006'] = {'color':'tan', 'name':'E. PTethys'}
target_plate_id_list['9004'] = {'color':'lightslategray', 'name':'S. Rheic'}
target_plate_id_list['9005'] = {'color':'slategray', 'name':'N. Rheic'}
#target_plate_id_list['430'] = {'color':'red', 'name':'Amuria'}
#target_plate_id_list['401'] = {'color':'indianred', 'name':'Siberia'}

#target_plate_id_list = ['901','902','911','926','919','908','982','983','701','715','801','802','201','501','101','301','102',
#                        '528','529','530','505','9010','9009','16111','305','9004','9005','9006','430','401']

time_step = 5
time_max = 400
#time_bins = np.arange(float(time_step)/2.,time_max,time_step)
time_bins = np.arange(0,410,5)

def get_ocean_area_time_series_for_plate(df,plate_id):
    
    mean_area = []
    for time in time_bins:
        #subset = df.where((df.time>=time) & (df.time<time+time_step)
        #                       & (df.plate_id==plate_id))
        subset = df.where((df.time==time) & (df.plate_id==plate_id))
        #if not np.isnan(np.nanmean(subset.plate_area - subset.continental_area)):
        #    mean_area.append(np.nanmedian(subset.plate_area - subset.continental_area))
        #else:
        #    mean_area.append(0.)
        mean_area.append(np.nansum(subset.plate_area - subset.continental_area))
        
    return mean_area


continental_area = []
for time in time_bins:
    #subset = df_M2016.where((df_M2016.time>=time) & (df_M2016.time<time+time_step))
    subset = df_M2016.where(df_M2016.time==time)
    continental_area.append(np.nansum(subset.continental_area))


stack_input = []
stack_input.append(continental_area)
for target_plate_id in target_plate_id_list:
    ocean_area_time_series = get_ocean_area_time_series_for_plate(df_M2016,int(target_plate_id))
    stack_input.append(ocean_area_time_series)

    

plt.figure(figsize=(10,6))

plt.stackplot(time_bins,np.array(stack_input),
              colors=['darkgrey'] + [target_plate_id_list[p]['color'] for p in target_plate_id_list])
#plt.plot(time_bins,np.array(stack_input).T)

plt.xlim(400,0)
plt.ylim(0,(pygplates.Earth.mean_radius_in_kms**2)*4*np.pi)
plt.legend(['Continents'] + [target_plate_id_list[p]['name'] for p in target_plate_id_list], 
           loc='upper left',bbox_to_anchor=(1.05, 0.05, 1, 0.9),ncol=2)
plt.grid()
plt.ylabel('Rift Length (km)')
plt.xlabel('Age (Ma)')
plt.title('Oceanic Plate Area')

plt.show()

In [ ]:
#print ['grey'] + [target_plate_id_list__[p] for p in target_plate_id_list__]
#print [p for p in target_plate_id_list.iterkeys()]

for p in target_plate_id_list:
    print target_plate_id_list[p]['name']

#target_plate_id_list = OrderedDict()
#target_plate_id_list['901'] = {'color':'navy', 'name':'Pacific'}
#target_plate_id_list['901']['color']

In [ ]:
https://github.com/siwill22/GPlatesClassStruggle/blob/master/test_VelocityField.ipynb